<a href="https://colab.research.google.com/github/IA-Programming/Probando-Mistral-7B-8bit/blob/main/Mistral7llamaindexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/10QPfcDt39uGciEDqdYBAbPBNZQDoC99O?usp=sharing)
[![Open In YouTube](https://img.shields.io/badge/Open%20In%20YouTube-FF0000?style=flat&logo=youtube&logoColor=white&labelColor=FF0000)](https://www.youtube.com/watch?v=kSLcedGSez8)


In [1]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 5.8 MB/s eta 0:00:00


In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 178.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423775 sha256=e6ff25c98dd3b0005bd241769c2a2f8b49ae7627aa75d2e98ed27aa328a73bc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-64qwbgh9/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


In [4]:
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.1/868.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [7]:
%%shell

# URL of the PDF file
pdf_url="https://www.versusarthritis.org/media/24901/fibromyalgia-information-booklet-july2021.pdf"

# Destination path where you want to save the PDF file
destination_path="/content/data"

# Use wget to download the PDF
wget -P "$destination_path" "$pdf_url"

# Check if the download was successful
if [ $? -eq 0 ]; then
    echo "PDF downloaded successfully."
else
    echo "Failed to download the PDF."
fi

--2023-10-08 01:44:01--  https://www.versusarthritis.org/media/24901/fibromyalgia-information-booklet-july2021.pdf
Resolving www.versusarthritis.org (www.versusarthritis.org)... 13.107.246.38, 13.107.213.38, 2620:1ec:bdf::38, ...
Connecting to www.versusarthritis.org (www.versusarthritis.org)|13.107.246.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2223460 (2.1M) [application/pdf]
Saving to: ‘/content/data/fibromyalgia-information-booklet-july2021.pdf’

fibromyalgia-inform 100%[===================>]   2.12M  2.34MB/s    in 0.9s    

2023-10-08 01:44:03 (2.34 MB/s) - ‘/content/data/fibromyalgia-information-booklet-july2021.pdf’ saved [2223460/2223460]

PDF downloaded successfully.


In [9]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [6]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:17, 234.90it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [10]:
!pip -q install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00


In [11]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

In [12]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("What is Fibromyalgia?")

In [15]:
print(response)

 Fibromyalgia is a long-term condition that causes pain and tenderness all over your body. It's thought to be caused by your nervous system in your brain and spine not being able to control or process pain signals from other parts of your body. The condition is also linked to poor sleep, feeling you can't concentrate or remember things, extreme and constant tiredness that doesn't improve with sleep or rest (known as fatigue), and many other symptoms that affect different parts of your body.


In [16]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

What are the symptoms of Fibromyalgia


Llama.generate: prefix-match hit


 The symptoms of Fibromyalgia include pain that spreads throughout your entire body, with certain parts such as your neck and back feeling particularly painful, tiredness, fatigue and generally feeling like you have no energy, sleeping badly, waking up feeling unrested, aching and stiffness, which often feels worse during or after you've been active.
What are various treatments for Fibromyalgia


Llama.generate: prefix-match hit


 There are several treatment options available for fibromyalgia, including medications, complementary therapies, and lifestyle changes. Medications can help reduce symptoms, but they cannot cure fibromyalgia. Complementary therapies such as massage, psychological therapy, and acupuncture may also be helpful for some people with fibromyalgia. Lifestyle changes such as getting enough good-quality sleep and exercising regularly are also important parts of treatment. It's important to speak with a healthcare provider before starting any new treatment options.
False


Llama.generate: prefix-match hit


 I apologize, but I am unable to determine if the given statement is false or not based on the provided context information. Could you please provide me with more specific details or a clearer question so that I can assist you better?


KeyboardInterrupt: ignored